In [176]:
# Load packages and settings
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 50)
import seaborn as sns


import matplotlib.pyplot as plt
%matplotlib widget
plt.rcParams['figure.figsize'] = (12,8)
plt.rcParams["image.cmap"] = "tab10"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)
fs_label = 15
parameters = {
                'axes.labelsize': fs_label,
                'axes.titlesize': fs_label+4,
                'xtick.labelsize': fs_label,
                'ytick.labelsize': fs_label, 
                'legend.fontsize': fs_label, 
                'lines.markersize': 10,
                'lines.linewidth': 3
             }
plt.rcParams.update(parameters)
%matplotlib widget
from matplotlib import cm # Colormaps
import matplotlib.colors as colors
# cmap = plt.cm.get_cmap('Dark2',len(ageGroups))

import locale
import matplotlib.dates as mdates
locale.setlocale(locale.LC_TIME,"Danish")
# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))

from matplotlib.colors import LinearSegmentedColormap


import os
# import csv
import math

from datetime import date


saveFigures = True
# saveFigures = False
print('saveFigures is set to: '+str(saveFigures))

print('Done loading packages')

# Define running mean functions
def rnMean(data,meanWidth):
    return np.convolve(data, np.ones(meanWidth)/meanWidth, mode='valid')
def rnTime(t,meanWidth):
    return t[math.floor(meanWidth/2):-math.ceil(meanWidth/2)+1]
    
    
# Define paths
rootdir_data = os.getcwd() +"\\..\\DanskeData\\" 

path_data = rootdir_data + "ssi_data\\"
path_dash = rootdir_data + "ssi_dashboard\\"
path_vacc = rootdir_data + "ssi_vacc\\"

path_figs = os.getcwd() +"\\..\\Figures\\" 

saveFigures is set to: True
Done loading packages


In [177]:

latestsubdir = list(os.walk(path_dash))[0][1][-1]
latestdir = path_dash + latestsubdir
latestdir

# dfAge = pd.read_csv(latestdir+'\\Regionalt_DB\\18_fnkt_alder_uge_testede_positive_nyindlagte.csv',delimiter=';',encoding='latin1')

dfCurAdmitted = pd.read_csv(latestdir+'\\Gennembruds_DB\\01_indlagte_pr_vaccinationsstatus.csv',delimiter=';',encoding='latin1')
df = pd.read_csv(latestdir+'\\Gennembruds_DB\\06_nyindlagte_pr_vaccinationsstatus_pr_aldersgrp_pr_uge.csv',delimiter=';',encoding='latin1')

df['Nyindlagte pr. 100.000'] = pd.to_numeric(df['Nyindlagte pr. 100.000'].str.replace(',','.'))

In [178]:
# curCount.values[0]
# df
# df[df.Vaccinationsstatus == 'Ikke vaccineret') & df.].loc[550:]
df[(df.Vaccinationsstatus == 'Ikke vaccineret') & (df.Aldersgruppe == '80+')].iloc[-15:]

,Uge,Aldersgruppe,Vaccinationsstatus,Antal nyindlagte,Befolkningstørrelse,Nyindlagte pr. 100.000
583,2021-W33,80+,Ikke vaccineret,2,10145,19.71
601,2021-W34,80+,Ikke vaccineret,6,10045,59.73
619,2021-W35,80+,Ikke vaccineret,9,9951,90.45
634,2021-W36,80+,Ikke vaccineret,3,9889,30.34
646,2021-W37,80+,Ikke vaccineret,0,9812,0.00
661,2021-W38,80+,Ikke vaccineret,2,9701,20.62
679,2021-W39,80+,Ikke vaccineret,0,9617,0.00
697,2021-W40,80+,Ikke vaccineret,1,9560,10.46
712,2021-W41,80+,Ikke vaccineret,8,9523,84.01
730,2021-W42,80+,Ikke vaccineret,4,9484,42.18


In [179]:
# Shorten data to use
firstWeek = 20
firstIndexToUse = df[df.Uge == '2021-W20'].index[0]
dfShort = df.loc[firstIndexToUse:]

dfVacc = dfShort[dfShort.Vaccinationsstatus == 'Forventet fuld effekt']

lastWeek = int(df.iloc[-1]['Uge'][-2:])

totNumWeeks = int(lastWeek-firstWeek)
weekRange = np.arange(firstWeek,lastWeek+1)

weekLabels = ['Uge\n'+str(x) for x in weekRange]
weekLabels = ['Uge '+str(x) for x in weekRange]

allAgeGroups = df.Aldersgruppe.unique()

fullMatVaccInci = np.zeros((totNumWeeks+1,len(allAgeGroups)))
fullMatVaccCount = np.zeros((totNumWeeks+1,len(allAgeGroups)))

for ageIndex in range(len(allAgeGroups)):
    curAgeGroup = allAgeGroups[ageIndex]

    agedf = dfVacc[(dfVacc.Aldersgruppe == curAgeGroup)]


    ageInci = []
    ageCount = []
    
    # for weekNum in np.arange(firstWeek,lastWeek+1):
    for weekIndex in range(totNumWeeks+1):
        weekNum = weekRange[weekIndex]
        thisdf = agedf[agedf.Uge == str('2021-W'+str(weekNum))]
        
        curInci = thisdf['Nyindlagte pr. 100.000']
        curCount = thisdf['Antal nyindlagte']
        
        if len(curInci) == 0:
            ageInci.append(0)
            ageCount.append(0)
        else:
            ageInci.append(curInci.values[0])
            ageCount.append(curCount.values[0])
            
    fullMatVaccInci[:,ageIndex] = ageInci
    fullMatVaccCount[:,ageIndex] = ageCount
            
# fig,ax1 = plt.subplots()
# ax1.plot(weekRange,ageCount)


# # list(dfVacc.groupby('Aldersgruppe'))
# thisdf = list(dfVacc.groupby('Aldersgruppe'))[1][1]

# curWeekNum = pd.to_numeric(thisdf.Uge.apply(lambda x: x[-2:]))

# thisdf
# # fig,ax1 = plt.subplots()
# # ax1.plot(curWeekNum,thisdf['Nyindlagte pr. 100.000'])

In [180]:
# Shorten data to use
firstWeek = 20
firstIndexToUse = df[df.Uge == '2021-W20'].index[0]
dfShort = df.loc[firstIndexToUse:]

dfUnvacc = dfShort[dfShort.Vaccinationsstatus == 'Ikke vaccineret']

lastWeek = int(df.iloc[-1]['Uge'][-2:])

totNumWeeks = int(lastWeek-firstWeek)
weekRange = np.arange(firstWeek,lastWeek+1)

weekLabels = ['Uge\n'+str(x) for x in weekRange]
weekLabels = ['Uge '+str(x) for x in weekRange]

allAgeGroups = df.Aldersgruppe.unique()

fullMatUnvaccInci = np.zeros((totNumWeeks+1,len(allAgeGroups)))
fullMatUnvaccCount = np.zeros((totNumWeeks+1,len(allAgeGroups)))

for ageIndex in range(len(allAgeGroups)):
    curAgeGroup = allAgeGroups[ageIndex]

    agedf = dfUnvacc[(dfUnvacc.Aldersgruppe == curAgeGroup)]


    ageInci = []
    ageCount = []
    
    # for weekNum in np.arange(firstWeek,lastWeek+1):
    for weekIndex in range(totNumWeeks+1):
        weekNum = weekRange[weekIndex]
        thisdf = agedf[agedf.Uge == str('2021-W'+str(weekNum))]
        
        curInci = thisdf['Nyindlagte pr. 100.000']
        curCount = thisdf['Antal nyindlagte']
        
        if len(curInci) == 0:
            ageInci.append(0)
            ageCount.append(0)
        else:
            ageInci.append(curInci.values[0])
            ageCount.append(curCount.values[0])
            
    fullMatUnvaccInci[:,ageIndex] = ageInci
    fullMatUnvaccCount[:,ageIndex] = ageCount
            
# fig,ax1 = plt.subplots()
# ax1.plot(weekRange,ageCount)


# # list(dfUnvacc.groupby('Aldersgruppe'))
# thisdf = list(dfUnvacc.groupby('Aldersgruppe'))[1][1]

# curWeekNum = pd.to_numeric(thisdf.Uge.apply(lambda x: x[-2:]))

# thisdf
# # fig,ax1 = plt.subplots()
# # ax1.plot(curWeekNum,thisdf['Nyindlagte pr. 100.000'])

In [181]:
# dfVacc
# list(dfVacc.groupby('Aldersgruppe'))
# thisdf = list(dfVacc.groupby('Aldersgruppe'))[1][1]

# curWeekNum = pd.to_numeric(thisdf.Uge.apply(lambda x: x[-2:]))

# thisdf
# fig,ax1 = plt.subplots()
# ax1.plot(curWeekNum,thisdf['Nyindlagte pr. 100.000'],'*-')¨

In [182]:

levelsToShow = [0,1,2,5,10,20,50,100]
levelsToShow = [0,5,10,20,50,100]
hex_colors = sns.light_palette('navy', n_colors=len(levelsToShow) , as_cmap=False).as_hex()
hex_colors = sns.light_palette("#4040ff", n_colors=len(levelsToShow) , as_cmap=False).as_hex()
# hex_colors = sns.light_palette('DarkBlue', n_colors=len(levelsToShow) , as_cmap=False).as_hex()
# hex_colors = sns.light_palette(("#ffffff","#0000ff","#000000"), n_colors=len(levelsToShow) , as_cmap=False).as_hex()
# hex_colors = sns.light_palette(, n_colors=len(levelsToShow) , as_cmap=False).as_hex()

# print(hex_colors) 

new_hex_colors = []
curVal = 0
# prevVal = 0
for k in range(len(levelsToShow)-1):
    # print(k)
    
    curHexColor = hex_colors[k]
    
    nextVal = levelsToShow[k+1] 
    curVal = levelsToShow[k] 
    for n in range(nextVal-curVal):
        new_hex_colors.append(curHexColor)

        
#     print(levelsToShow[k])
    
#     print(nextVal)
#     if curVal < nextVal:
#         new_hex_colors.append(curHexColor)
#         curVal = curVal + 1
#         print(curVal)
#     # prevVal = curVal
# print(new_hex_colors)
hex_colors

['#f0f0f3', '#cdcdf5', '#aaaaf8', '#8787fa', '#6363fd', '#4040ff']

In [183]:
fig,ax1 = plt.subplots(figsize=(14,8))


# hex_colors = sns.light_palette('navy', n_colors=len(levelsToShow) , as_cmap=False).as_hex()
# myColors = ((0.8, 0.0, 0.0, 1.0), (0.0, 0.8, 0.0, 1.0), (0.0, 0.0, 0.8, 1.0))
# cmap = LinearSegmentedColormap.from_list('Custom', hex_colors, len(hex_colors))
cmap = LinearSegmentedColormap.from_list('Custom', new_hex_colors, len(new_hex_colors))

g = sns.heatmap(fullMatVaccInci.transpose(),
                ax=ax1,
                annot=True,
                fmt='1.0f',
                vmax=100,
                xticklabels=weekLabels,
                yticklabels=allAgeGroups,
                cmap=cmap,
                )
g.invert_yaxis()
plt.yticks(rotation=0)

cbar = ax1.collections[0].colorbar
cbar.set_ticks(levelsToShow)

# plt.title('Vaccinated, hospitalizations per 100.000')
plt.title('Nye indlæggelser per 100.000 vaccinerede i aldersgruppen')
plt.tight_layout()
if saveFigures:
    plt.savefig(path_figs+'HeatmapBreakthrough_Vaccinated')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [184]:
fig,ax1 = plt.subplots(figsize=(14,8))


# levelsToShow = [0,1,5,10,20,50]
# hex_colors = sns.light_palette('navy', n_colors=len(levelsToShow) , as_cmap=False).as_hex()
# # myColors = ((0.8, 0.0, 0.0, 1.0), (0.0, 0.8, 0.0, 1.0), (0.0, 0.0, 0.8, 1.0))
# cmap = LinearSegmentedColormap.from_list('Custom', hex_colors, len(hex_colors))

# g = sns.heatmap(fullMatUnvaccInci.transpose(),
#                 ax=ax1,
#                 annot=True,
#                 fmt='1.0f',
#                 vmax=30,
#                 xticklabels=weekLabels,
#                 yticklabels=allAgeGroups,
#                 cmap=cmap,
#                 )
# g.invert_yaxis()
# plt.yticks(rotation=0)


cmap = LinearSegmentedColormap.from_list('Custom', new_hex_colors, len(new_hex_colors))

g = sns.heatmap(fullMatUnvaccInci.transpose(),
                ax=ax1,
                annot=True,
                fmt='1.0f',
                vmax=100,
                xticklabels=weekLabels,
                yticklabels=allAgeGroups,
                cmap=cmap,
                )
g.invert_yaxis()
plt.yticks(rotation=0)

cbar = ax1.collections[0].colorbar
cbar.set_ticks(levelsToShow)

# plt.title('Unvaccinated, hospitalizations per 100.000')
# plt.title('Uvaccinerede, nye indlæggelser per 100.000')
plt.title('Nye indlæggelser per 100.000 uvaccinerede i aldersgruppen')
plt.tight_layout()

if saveFigures:
    plt.savefig(path_figs+'HeatmapBreakthrough_Unvaccinated')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [185]:
fig,(ax1,ax2) = plt.subplots(2,1,figsize=(14,8))


cmap = LinearSegmentedColormap.from_list('Custom', new_hex_colors, len(new_hex_colors))

g = sns.heatmap(fullMatVaccInci.transpose(),
                ax=ax1,
                annot=True,
                fmt='1.0f',
                vmax=100,
                xticklabels=weekLabels,
                yticklabels=allAgeGroups,
                cmap=cmap,
                # cbar=False
                )
g.invert_yaxis()
plt.yticks(rotation=0)

cbar = ax1.collections[0].colorbar
cbar.set_ticks(levelsToShow)
fig.axes[2].set_visible(False)

g = sns.heatmap(fullMatUnvaccInci.transpose(),
                ax=ax2,
                annot=True,
                fmt='1.0f',
                vmax=100,
                xticklabels=weekLabels,
                yticklabels=allAgeGroups,
                cmap=cmap,
                )
g.invert_yaxis()
plt.yticks(rotation=0)

cbar = ax2.collections[0].colorbar
cbar.set_ticks(levelsToShow)
# cbar.set_ticklabels(levelsToShow,textsize=12)
cax = plt.gcf().axes[-1]
cax.tick_params(labelsize=10)

ax1.set_title('Nye indlæggelser per\n100.000 vaccinerede i aldersgruppen')
ax2.set_title('Nye indlæggelser per\n100.000 uvaccinerede i aldersgruppen')

fig.tight_layout()

if saveFigures:
    plt.savefig(path_figs+'HeatmapBreakthrough_BothLong')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [186]:
# Shorten data to use - EVEN SHORTER!
firstWeek = 32
firstIndexToUse = df[df.Uge == f'2021-W{firstWeek}'].index[0]
dfShort = df.loc[firstIndexToUse:]

dfUnvacc = dfShort[dfShort.Vaccinationsstatus == 'Ikke vaccineret']

lastWeek = int(df.iloc[-1]['Uge'][-2:])

totNumWeeks = int(lastWeek-firstWeek)
weekRange = np.arange(firstWeek,lastWeek+1)

weekLabels = ['Uge\n'+str(x) for x in weekRange]
weekLabels = ['Uge '+str(x) for x in weekRange]

allAgeGroups = df.Aldersgruppe.unique()

fullMatUnvaccInci = np.zeros((totNumWeeks+1,len(allAgeGroups)))
fullMatUnvaccCount = np.zeros((totNumWeeks+1,len(allAgeGroups)))

for ageIndex in range(len(allAgeGroups)):
    curAgeGroup = allAgeGroups[ageIndex]

    agedf = dfUnvacc[(dfUnvacc.Aldersgruppe == curAgeGroup)]


    ageInci = []
    ageCount = []
    
    # for weekNum in np.arange(firstWeek,lastWeek+1):
    for weekIndex in range(totNumWeeks+1):
        weekNum = weekRange[weekIndex]
        thisdf = agedf[agedf.Uge == str('2021-W'+str(weekNum))]
        
        curInci = thisdf['Nyindlagte pr. 100.000']
        curCount = thisdf['Antal nyindlagte']
        
        if len(curInci) == 0:
            ageInci.append(0)
            ageCount.append(0)
        else:
            ageInci.append(curInci.values[0])
            ageCount.append(curCount.values[0])
            
    fullMatUnvaccInci[:,ageIndex] = ageInci
    fullMatUnvaccCount[:,ageIndex] = ageCount
            
fullMatVaccInci = np.zeros((totNumWeeks+1,len(allAgeGroups)))
fullMatVaccCount = np.zeros((totNumWeeks+1,len(allAgeGroups)))

for ageIndex in range(len(allAgeGroups)):
    curAgeGroup = allAgeGroups[ageIndex]

    agedf = dfVacc[(dfVacc.Aldersgruppe == curAgeGroup)]


    ageInci = []
    ageCount = []
    
    # for weekNum in np.arange(firstWeek,lastWeek+1):
    for weekIndex in range(totNumWeeks+1):
        weekNum = weekRange[weekIndex]
        thisdf = agedf[agedf.Uge == str('2021-W'+str(weekNum))]
        
        curInci = thisdf['Nyindlagte pr. 100.000']
        curCount = thisdf['Antal nyindlagte']
        
        if len(curInci) == 0:
            ageInci.append(0)
            ageCount.append(0)
        else:
            ageInci.append(curInci.values[0])
            ageCount.append(curCount.values[0])
            
    fullMatVaccInci[:,ageIndex] = ageInci
    fullMatVaccCount[:,ageIndex] = ageCount

In [187]:
fig,(ax1,ax2) = plt.subplots(1,2,figsize=(14,8))


cmap = LinearSegmentedColormap.from_list('Custom', new_hex_colors, len(new_hex_colors))

g = sns.heatmap(fullMatVaccInci.transpose(),
                ax=ax1,
                annot=True,
                fmt='1.0f',
                vmax=100,
                xticklabels=weekLabels,
                yticklabels=allAgeGroups,
                cmap=cmap,
                # cbar=False
                )
g.invert_yaxis()
plt.yticks(rotation=0)

cbar = ax1.collections[0].colorbar
cbar.set_ticks(levelsToShow)
fig.axes[2].set_visible(False)

g = sns.heatmap(fullMatUnvaccInci.transpose(),
                ax=ax2,
                annot=True,
                fmt='1.0f',
                vmax=100,
                xticklabels=weekLabels,
                yticklabels=allAgeGroups,
                cmap=cmap,
                )
g.invert_yaxis()
plt.yticks(rotation=0)

cbar = ax2.collections[0].colorbar
cbar.set_ticks(levelsToShow)

ax1.set_title('Nye indlæggelser per\n100.000 vaccinerede i aldersgruppen')
ax2.set_title('Nye indlæggelser per\n100.000 uvaccinerede i aldersgruppen')

fig.tight_layout()

if saveFigures:
    plt.savefig(path_figs+'HeatmapBreakthrough_Both')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [188]:
fig,(ax1,ax2) = plt.subplots(1,2,figsize=(14,8))


cmap = LinearSegmentedColormap.from_list('Custom', new_hex_colors, len(new_hex_colors))

g = sns.heatmap(fullMatVaccInci.transpose(),
                ax=ax1,
                # annot=True,
                # fmt='1.0f',
                vmax=100,
                xticklabels=weekLabels,
                yticklabels=allAgeGroups,
                cmap=cmap,
                # cbar=False
                )
g.invert_yaxis()
plt.yticks(rotation=0)

cbar = ax1.collections[0].colorbar
cbar.set_ticks(levelsToShow)
fig.axes[2].set_visible(False)

g = sns.heatmap(fullMatUnvaccInci.transpose(),
                ax=ax2,
                # annot=True,
                # fmt='1.0f',
                vmax=100,
                xticklabels=weekLabels,
                yticklabels=allAgeGroups,
                cmap=cmap,
                )
g.invert_yaxis()
plt.yticks(rotation=0)

cbar = ax2.collections[0].colorbar
cbar.set_ticks(levelsToShow)

ax1.set_title('Nye indlæggelser per\n100.000 vaccinerede i aldersgruppen')
ax2.set_title('Nye indlæggelser per\n100.000 uvaccinerede i aldersgruppen')

fig.tight_layout()

if saveFigures:
    plt.savefig(path_figs+'HeatmapBreakthrough_Both_NoText')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Try with extended labels

In [189]:
# Shorten data to use - EVEN SHORTER!
firstWeek = 32
firstIndexToUse = df[df.Uge == f'2021-W{firstWeek}'].index[0]
dfShort = df.loc[firstIndexToUse:]

dfUnvacc = dfShort[dfShort.Vaccinationsstatus == 'Ikke vaccineret']

lastWeek = int(df.iloc[-1]['Uge'][-2:])

totNumWeeks = int(lastWeek-firstWeek)
weekRange = np.arange(firstWeek,lastWeek+1)

weekLabels = ['Uge\n'+str(x) for x in weekRange]
weekLabels = ['Uge '+str(x) for x in weekRange]

allAgeGroups = df.Aldersgruppe.unique()

fullMatUnvaccInci = np.zeros((totNumWeeks+1,len(allAgeGroups)))
fullMatUnvaccCount = np.zeros((totNumWeeks+1,len(allAgeGroups)))
fullMatUnvaccPop = np.zeros((totNumWeeks+1,len(allAgeGroups)))

for ageIndex in range(len(allAgeGroups)):
    curAgeGroup = allAgeGroups[ageIndex]

    agedf = dfUnvacc[(dfUnvacc.Aldersgruppe == curAgeGroup)]


    ageInci = []
    ageCount = []
    agePop = []

    prevPop = 0
    
    # for weekNum in np.arange(firstWeek,lastWeek+1):
    for weekIndex in range(totNumWeeks+1):
        weekNum = weekRange[weekIndex]
        thisdf = agedf[agedf.Uge == str('2021-W'+str(weekNum))]
        
        curInci = thisdf['Nyindlagte pr. 100.000']
        curCount = thisdf['Antal nyindlagte']
        curPop = thisdf['Befolkningstørrelse']
        
        if len(curInci) == 0:
            ageInci.append(0)
            ageCount.append(0)
            # agePop.append(0)
            agePop.append(prevPop)
        else:
            ageInci.append(curInci.values[0])
            ageCount.append(curCount.values[0])
            agePop.append(curPop.values[0])
            prevPop = curPop.values[0]
            
    fullMatUnvaccInci[:,ageIndex] = ageInci
    fullMatUnvaccCount[:,ageIndex] = ageCount
    fullMatUnvaccPop[:,ageIndex] = agePop

fullMatVaccInci = np.zeros((totNumWeeks+1,len(allAgeGroups)))
fullMatVaccCount = np.zeros((totNumWeeks+1,len(allAgeGroups)))
fullMatVaccPop = np.zeros((totNumWeeks+1,len(allAgeGroups)))

for ageIndex in range(len(allAgeGroups)):
    curAgeGroup = allAgeGroups[ageIndex]

    agedf = dfVacc[(dfVacc.Aldersgruppe == curAgeGroup)]


    ageInci = []
    ageCount = []
    agePop = []

    prevPop = 0
    
    # for weekNum in np.arange(firstWeek,lastWeek+1):
    for weekIndex in range(totNumWeeks+1):
        weekNum = weekRange[weekIndex]
        thisdf = agedf[agedf.Uge == str('2021-W'+str(weekNum))]
        
        curInci = thisdf['Nyindlagte pr. 100.000']
        curCount = thisdf['Antal nyindlagte']
        curPop = thisdf['Befolkningstørrelse']
        
        if len(curInci) == 0:
            ageInci.append(0)
            ageCount.append(0)
            # agePop.append(0)
            agePop.append(prevPop)
        else:
            ageInci.append(curInci.values[0])
            ageCount.append(curCount.values[0])
            agePop.append(curPop.values[0])
            prevPop = curPop.values[0]
            
    fullMatVaccInci[:,ageIndex] = ageInci
    fullMatVaccCount[:,ageIndex] = ageCount
    fullMatVaccPop[:,ageIndex] = agePop

In [190]:
fullMatVaccPop

array([[   1942.,  160321.,  330105., 1514850.,  855895.,  259696.],
       [   7993.,  188326.,  459429., 1598392.,  857147.,  259882.],
       [  27811.,  204461.,  684227., 1652589.,  858605.,  260127.],
       [  66250.,  214281.,  872675., 1683318.,  859888.,  260355.],
       [ 102968.,  214281.,  978257., 1698135.,  860685.,  260522.],
       [ 102968.,  214281., 1036450., 1708105.,  861300.,  260621.],
       [ 102968.,  226727., 1063648., 1714150.,  861502.,  260390.],
       [ 167636.,  228851., 1082827., 1718883.,  861608.,  260096.],
       [ 176830.,  230547., 1097398., 1722647.,  861699.,  259756.],
       [ 176830.,  231751., 1107657., 1725447.,  861675.,  259379.],
       [ 186959.,  232646., 1114225., 1727406.,  861579.,  258913.],
       [ 190118.,  233321., 1119818., 1729138.,  861458.,  258408.],
       [ 192505.,  233786., 1124362., 1730456.,  861302.,  257963.],
       [ 195013.,  234276., 1128717., 1731795.,  861096.,  257433.],
       [ 197141.,  234769., 113316

In [191]:
plotdf = pd.DataFrame(fullMatVaccInci.transpose())
annodf = pd.DataFrame(fullMatVaccCount.transpose())
annodf2 = pd.DataFrame(fullMatVaccPop.transpose())
annodf3 = pd.DataFrame(fullMatVaccCount.transpose())
# display(plotdf)
# print(plotdf.shape)
# display(annodf)
# print(annodf.shape)
# for k in annodf.columns:
#     annodf[k] = 'Antal: '+ annodf[k].astype(int).astype(str)
# for k in annodf2.columns:
#     annodf2[k] = 'Pop: '+ annodf2[k].astype(int).astype(str)
# display(annodf)
# display(annodf2)

for k in annodf3.columns:
    annodf3[k] = annodf[k].astype(int).astype(str) + ' ud af\n' + annodf2[k].astype(int).astype(str)
annodf3

plotdf_unvacc = pd.DataFrame(fullMatUnvaccInci.transpose())
annodf_unvacc = pd.DataFrame(fullMatUnvaccCount.transpose())
annodf2_unvacc = pd.DataFrame(fullMatUnvaccPop.transpose())
annodf3_unvacc = pd.DataFrame(fullMatUnvaccCount.transpose())

for k in annodf3_unvacc.columns:
    annodf3_unvacc[k] = annodf_unvacc[k].astype(int).astype(str) + ' ud af\n' + annodf2_unvacc[k].astype(int).astype(str)
annodf3_unvacc



,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,3 ud af\n134544,2 ud af\n120285,3 ud af\n108897,3 ud af\n100863,1 ud af\n95682,0 ud af\n95682,0 ud af\n95682,0 ud af\n88861,2 ud af\n87968,0 ud af\n87968,1 ud af\n86384,2 ud af\n84942,3 ud af\n82653,2 ud af\n78747,2 ud af\n73370,2 ud af\n69615
1,3 ud af\n48950,6 ud af\n46571,6 ud af\n44478,2 ud af\n42836,0 ud af\n42836,0 ud af\n42836,1 ud af\n40561,0 ud af\n40385,2 ud af\n40156,1 ud af\n39947,3 ud af\n39714,1 ud af\n39431,0 ud af\n39046,2 ud af\n38183,2 ud af\n36529,1 ud af\n35199
2,55 ud af\n402259,70 ud af\n382043,57 ud af\n365771,51 ud af\n354228,34 ud af\n346822,13 ud af\n341185,12 ud af\n337606,13 ud af\n335301,23 ud af\n333584,25 ud af\n331706,34 ud af\n329482,43 ud af\n326269,66 ud af\n321885,49 ud af\n313255,63 ud af\n299745,72 ud af\n289824
3,39 ud af\n184796,52 ud af\n179701,38 ud af\n175100,47 ud af\n171610,37 ud af\n169128,24 ud af\n167105,12 ud af\n165713,18 ud af\n164680,21 ud af\n163848,30 ud af\n163041,34 ud af\n162248,43 ud af\n161142,50 ud af\n159674,46 ud af\n156595,71 ud af\n151643,71 ud af\n148275
4,4 ud af\n32464,6 ud af\n32056,12 ud af\n31605,10 ud af\n31191,6 ud af\n30914,1 ud af\n30656,7 ud af\n30418,6 ud af\n30197,4 ud af\n30077,5 ud af\n29982,7 ud af\n29868,13 ud af\n29702,17 ud af\n29515,16 ud af\n29178,19 ud af\n28722,27 ud af\n28360
5,3 ud af\n10224,2 ud af\n10145,6 ud af\n10045,9 ud af\n9951,3 ud af\n9889,0 ud af\n9812,2 ud af\n9701,0 ud af\n9617,1 ud af\n9560,8 ud af\n9523,4 ud af\n9484,8 ud af\n9430,8 ud af\n9379,6 ud af\n9278,13 ud af\n9141,13 ud af\n9036


In [192]:
fig,ax1 = plt.subplots(figsize=(14,8))



cmap = LinearSegmentedColormap.from_list('Custom', new_hex_colors, len(new_hex_colors))

g = sns.heatmap(data=plotdf,
                ax=ax1,
                annot=annodf,
                annot_kws={'va':'top',"fontsize":7},
                fmt='1.0f',
                vmax=100,
                xticklabels=weekLabels,
                yticklabels=allAgeGroups,
                cmap=cmap,
                cbar=False,
                )
                

g.invert_yaxis()
# plt.yticks(rotation=0)

# cbar = ax1.collections[0].colorbar
# cbar.set_ticks(levelsToShow)

g = sns.heatmap(data=plotdf,
                ax=ax1,
                annot=plotdf,
                annot_kws={'va':'bottom'},
                fmt='1.0f',
                vmax=100,
                xticklabels=weekLabels,
                yticklabels=allAgeGroups,
                cmap=cmap,
                )
g.invert_yaxis()
plt.yticks(rotation=0)

cbar = ax1.collections[1].colorbar
cbar.set_ticks(levelsToShow)
# cbar = ax1.collections[0].colorbar
# cbar.set_ticks(levelsToShow)
# fig.axes[2].set_visible(False)

plt.title('Nye indlæggelser per 100.000 vaccinerede i aldersgruppen')
plt.tight_layout()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [193]:
fig,ax1 = plt.subplots(figsize=(14,8))



cmap = LinearSegmentedColormap.from_list('Custom', new_hex_colors, len(new_hex_colors))

g = sns.heatmap(data=plotdf,
                ax=ax1,
                annot=annodf3,
                annot_kws={'va':'top',"fontsize":7},
                fmt='',
                vmax=100,
                xticklabels=weekLabels,
                yticklabels=allAgeGroups,
                cmap=cmap,
                cbar=False,
                )
                

g.invert_yaxis()

g = sns.heatmap(data=plotdf,
                ax=ax1,
                annot=plotdf,
                annot_kws={'va':'bottom',"fontsize":13,'fontweight':'bold'},
                fmt='1.0f',
                vmax=100,
                xticklabels=weekLabels,
                yticklabels=allAgeGroups,
                cmap=cmap,
                )
g.invert_yaxis()
plt.yticks(rotation=0)

cbar = ax1.collections[1].colorbar
cbar.set_ticks(levelsToShow)
# cbar = ax1.collections[0].colorbar
# cbar.set_ticks(levelsToShow)
# fig.axes[2].set_visible(False)

plt.title('Nye indlæggelser per 100.000 vaccinerede i aldersgruppen')
plt.tight_layout()

if saveFigures:
    plt.savefig(path_figs+'HeatmapBreakthrough_Vaccinated_Detailed')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [194]:
fig,ax1 = plt.subplots(figsize=(14,8))



cmap = LinearSegmentedColormap.from_list('Custom', new_hex_colors, len(new_hex_colors))

g = sns.heatmap(data=plotdf_unvacc,
                ax=ax1,
                annot=annodf3_unvacc,
                annot_kws={'va':'top',"fontsize":7},
                fmt='',
                vmax=100,
                xticklabels=weekLabels,
                yticklabels=allAgeGroups,
                cmap=cmap,
                cbar=False,
                )
                

g.invert_yaxis()

g = sns.heatmap(data=plotdf_unvacc,
                ax=ax1,
                annot=plotdf_unvacc,
                annot_kws={'va':'bottom',"fontsize":13,'fontweight':'bold'},
                fmt='1.0f',
                vmax=100,
                xticklabels=weekLabels,
                yticklabels=allAgeGroups,
                cmap=cmap,
                )
g.invert_yaxis()
plt.yticks(rotation=0)

cbar = ax1.collections[1].colorbar
cbar.set_ticks(levelsToShow)
# cbar = ax1.collections[0].colorbar
# cbar.set_ticks(levelsToShow)
# fig.axes[2].set_visible(False)

plt.title('Nye indlæggelser per 100.000 uvaccinerede i aldersgruppen')
plt.tight_layout()

if saveFigures:
    plt.savefig(path_figs+'HeatmapBreakthrough_Unvaccinated_Detailed')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [195]:
fig,(ax1,ax2) = plt.subplots(1,2,figsize=(14,8))


cmap = LinearSegmentedColormap.from_list('Custom', new_hex_colors, len(new_hex_colors))

g = sns.heatmap(data=plotdf,
                ax=ax1,
                annot=annodf3,
                annot_kws={'va':'top',"fontsize":5},
                fmt='',
                vmax=100,
                xticklabels=weekLabels,
                yticklabels=allAgeGroups,
                cmap=cmap,
                cbar=False,
                )
                

g.invert_yaxis()

g = sns.heatmap(data=plotdf,
                ax=ax1,
                annot=plotdf,
                annot_kws={'va':'bottom',"fontsize":13,'fontweight':'bold'},
                fmt='1.0f',
                vmax=100,
                xticklabels=weekLabels,
                yticklabels=allAgeGroups,
                cmap=cmap,
                )
g.invert_yaxis()
plt.yticks(rotation=0)

cbar = ax1.collections[1].colorbar
cbar.set_ticks(levelsToShow)
fig.axes[2].set_visible(False)

g = sns.heatmap(fullMatUnvaccInci.transpose(),
                ax=ax2,
                annot=True,
                fmt='1.0f',
                vmax=100,
                xticklabels=weekLabels,
                yticklabels=allAgeGroups,
                cmap=cmap,
                )
g.invert_yaxis()
plt.yticks(rotation=0)

cbar = ax2.collections[0].colorbar
cbar.set_ticks(levelsToShow)

ax1.set_title('Nye indlæggelser per\n100.000 vaccinerede i aldersgruppen')
ax2.set_title('Nye indlæggelser per\n100.000 uvaccinerede i aldersgruppen')

fig.tight_layout()

# if saveFigures:
#     plt.savefig(path_figs+'HeatmapBreakthrough_Both')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [196]:
# fullMatVaccInci.transpose()

# plotdf = pd.DataFrame()
# annodf = pd.DataFrame()
# annodf2 = pd.DataFrame()
# for k in range(len(allAgeGroups)):
#     plotdf[allAgeGroups[k]] = fullMatVaccInci.transpose()[k]
#     # annodf[allAgeGroups[k]] = fullMatVaccInci.transpose()[k]



#     thisdf = dfShort[(dfShort.Aldersgruppe == allAgeGroups[k]) & (dfShort.Vaccinationsstatus == 'Forventet fuld effekt')]
#     popSize = thisdf['Befolkningstørrelse'].values
#     numHosp = thisdf['Antal nyindlagte'].values
    
#     # print(popSize)
#     # annodf[allAgeGroups[k]] = popSize
#     # annodf2[allAgeGroups[k]] = numHosp
# display(thisdf)
# plotdf

In [197]:
thisdfUnvacc = dfUnvacc[dfUnvacc.Uge == '2021-W47']
thisdfVacc = dfVacc[dfVacc.Uge == '2021-W47'] 
thisdfVacc

,Uge,Aldersgruppe,Vaccinationsstatus,Antal nyindlagte,Befolkningstørrelse,Nyindlagte pr. 100.000
804,2021-W47,12-15,Forventet fuld effekt,1,199075,0.50
807,2021-W47,16-19,Forventet fuld effekt,3,235374,1.27
810,2021-W47,20-39,Forventet fuld effekt,33,1138264,2.90
813,2021-W47,40-64,Forventet fuld effekt,72,1734684,4.15
816,2021-W47,65-79,Forventet fuld effekt,106,860734,12.32
819,2021-W47,80+,Forventet fuld effekt,70,256461,27.29


In [198]:
totHospUnva = thisdfUnvacc['Antal nyindlagte'].sum()
totHospVacc = thisdfVacc['Antal nyindlagte'].sum()

avgPopUnva = int(np.round(thisdfUnvacc.Befolkningstørrelse.sum()))
avgPopVacc = int(np.round(thisdfVacc.Befolkningstørrelse.sum()))

print(f'{totHospUnva} indlæggelser ud af omkring {avgPopUnva} uvaccinerede')
print(f'{totHospVacc} indlæggelser ud af omkring {avgPopVacc} vaccinerede')

print(totHospUnva/100)
print(avgPopUnva/100)
print(totHospVacc/100)
print(avgPopVacc/100)

186 indlæggelser ud af omkring 580309 uvaccinerede
285 indlæggelser ud af omkring 4424592 vaccinerede
1.86
5803.09
2.85
44245.92


In [199]:
totHospUnva = agedfUnvacc['Antal nyindlagte'].sum()
totHospVacc = agedfVacc['Antal nyindlagte'].sum()

avgPopUnva = int(np.round(agedfUnvacc.Befolkningstørrelse.mean()))
avgPopVacc = int(np.round(agedfVacc.Befolkningstørrelse.mean()))

print(f'{totHospUnva} indlæggelser ud af omkring {avgPopUnva} uvaccinerede 80+')
print(f'{totHospVacc} indlæggelser ud af omkring {avgPopVacc} vaccinerede 80+')

print(totHospUnva/100)
print(avgPopUnva/100)
print(totHospVacc/100)
print(avgPopVacc/100)

86 indlæggelser ud af omkring 9638 uvaccinerede 80+
486 indlæggelser ud af omkring 259184 vaccinerede 80+
0.86
96.38
4.86
2591.84


In [209]:
UH = int(np.round(totHospUnva/100))
U = int(np.round(avgPopUnva/100))
VH = int(np.round(totHospVacc/100))
V = int(np.round(avgPopVacc/100))

# plt.figure()


# m1 = [1,1]
# v1 = [0.4,0.4]
# k1 = V

# data1 = v1 * np.random.randn(k1,2) + m1

rawData = np.random.rand(V-VH,2)

data1 = rawData.copy()
curRadius = 1
data1[:,0] = curRadius* np.sin(2*np.pi*rawData[:,0])*np.sqrt(rawData[:,1])
data1[:,1] = curRadius* np.cos(2*np.pi*rawData[:,0])*np.sqrt(rawData[:,1])

ts = np.linspace(-1,1,3000)
xs = curRadius * np.sin(2*np.pi*ts)
ys = curRadius * np.cos(2*np.pi*ts)

rawData2 = np.random.rand(U-UH,2)

data2 = rawData2.copy()
curRadius2 = 0.25
data2[:,0] = curRadius2* np.sin(2*np.pi*rawData2[:,0])*np.sqrt(rawData2[:,1]) - curRadius - curRadius2
data2[:,1] = curRadius2* np.cos(2*np.pi*rawData2[:,0])*np.sqrt(rawData2[:,1])

# data1[data1[:,0] < -2.75,0] = data1[data1[:,0] < -2.75,0] + 0.2

xs2 = curRadius2 * np.sin(2*np.pi*ts)- curRadius - curRadius2
ys2 = curRadius2 * np.cos(2*np.pi*ts)

plt.figure(figsize=(30,30))
# plt.plot(rawData[:,0],rawData[:,1],'.')
# plt.plot(xs,ys,'m:',linewidth=1)
plt.plot(data1[:,0],data1[:,1],'m.',markersize=15)
plt.plot(data2[:,0],data2[:,1],'g.',markersize=15)

plt.axis('equal')

if saveFigures:
    plt.savefig(path_figs+'HospCircle80+')
    
    
plt.plot(xs,ys,'m:',linewidth=1)
plt.plot(xs2,ys2,'g:',linewidth=1)

if saveFigures:
    plt.savefig(path_figs+'HospCircle80+_Boundary')

# plt.figure()
# # plt.plot(rawData[:,0],rawData[:,1],'.')
# plt.plot(xs,ys,'k',linewidth=1)
# plt.plot(data1[:,0],data1[:,1],'.',markersize=3)
# zoomLevel = 0.2
# plt.xlim([-3-zoomLevel,-3+zoomLevel])
# plt.ylim([-zoomLevel,zoomLevel])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [201]:

def circle_points(r, n):
    circles = []
    for r, n in zip(r, n):
        t = np.linspace(0, 2*np.pi, n, endpoint=False)
        x = r * np.cos(t)
        y = r * np.sin(t)
        circles.append(np.c_[x, y])
    return circles

In [202]:
r = [0, 0.1, 0.2]
n = [1, 10, 20]
circles = circle_points(r, n)
fig, ax = plt.subplots()
for circle in circles:
    ax.scatter(circle[:, 0], circle[:, 1])
ax.set_aspect('equal')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Old stuff and other stuff

In [203]:

# from matplotlib.colors import ListedColormap
# cmap = ListedColormap(['red','black','yellow', 'blue'])
# # colorbar.set_ticklabels(['A', 'B', 'C', 'NA'])

# fig,ax1 = plt.subplots(figsize=(16,8))
# g = sns.heatmap(fullMatVaccInci.transpose(),
#                 ax=ax1,
#                 annot=True,
#                 fmt='1.0f',
#                 xticklabels=weekLabels,
#                 yticklabels=allAgeGroups,
#                 cmap=cmap,
#                 )
# g.invert_yaxis()
# plt.yticks(rotation=0)

In [204]:


# # curMax = fullMatVaccInci.max()


# levelsToShow = [0,1,5,10,20,50]
# # boundaries = np.array([0.0,5,10,20,50]) / 50
# # # boundaries = [0.0, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 1.0]  # custom boundaries
# # # boundaries = np.arange(0,1,10)
# # # hex_colors = sns.light_palette('navy', n_colors=len(boundaries) * 2 + 2, as_cmap=False).as_hex()
# # hex_colors = sns.light_palette('navy', n_colors=len(boundaries) + 2, as_cmap=False).as_hex()
# # hex_colors = [hex_colors[i] for i in range(0, len(hex_colors), 1)]
# # colors=list(zip(boundaries, hex_colors))
# # custom_color_map = LinearSegmentedColormap.from_list(
# #     name='custom_navy',
# #     colors=colors,
# # )

# hex_colors = sns.light_palette('navy', n_colors=len(levelsToShow) + 1, as_cmap=False).as_hex()
# # myColors = ((0.8, 0.0, 0.0, 1.0), (0.0, 0.8, 0.0, 1.0), (0.0, 0.0, 0.8, 1.0))
# cmap = LinearSegmentedColormap.from_list('Custom', hex_colors, len(hex_colors))


# fig,ax1 = plt.subplots(figsize=(16,8))
# g = sns.heatmap(fullMatVaccInci.transpose(),
#                 ax=ax1,
#                 annot=True,
#                 fmt='1.1f',
#                 vmax=50,
#                 xticklabels=weekLabels,
#                 yticklabels=allAgeGroups,
#                 cmap=cmap,
#                 # cbar_kws={'ticks': [0.0, 0.2, 0.4, 0.5, 0.7, 0.8, 1.0]}
#                 )
# g.invert_yaxis()

# # colorbar = ax1.collections[0].colorbar
# # colorbar.set_ticks(levelsToShow)
# # colorbar.set_ticklabels(['B', 'A', 'C'])
# plt.yticks(rotation=0)

In [205]:

# curdf = df 
curdf = dfShort[(dfShort.Aldersgruppe == '80+') & (dfShort.Vaccinationsstatus == 'Ikke vaccineret')]
curdf = dfShort[(dfShort.Aldersgruppe == '80+') & (dfShort.Vaccinationsstatus == 'Forventet fuld effekt')]
curdf

curWeekNum = curdf.Uge.apply(lambda x: int(x[6:]))

curAge = '20-39'
for curAge in dfShort.Aldersgruppe.unique():
    curAgeLabel = 'Personer i aldersgruppen '+curAge
    agedf = dfShort[(dfShort.Aldersgruppe == curAge)]

    curSums = agedf.groupby('Uge').sum()

    fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)


    # ax1.plot(curWeekNum,curSums['Antal nyindlagte'])

    agedfVacc = agedf[agedf.Vaccinationsstatus == 'Forventet fuld effekt']
    agedfUnvacc = agedf[agedf.Vaccinationsstatus == 'Ikke vaccineret']
    # ax1.fill_between(curWeekNum,agedfUnvacc['Antal nyindlagte'].values,color='xkcd:light red')
    # ax1.fill_between(curWeekNum,agedfVacc['Antal nyindlagte'].values,agedfUnvacc['Antal nyindlagte'].values,color='b')

    curWeekNum = agedfVacc.Uge.apply(lambda x: int(x[6:]))

    hospUn = agedfUnvacc['Antal nyindlagte'].values
    hospVa = agedfVacc['Antal nyindlagte'].values
    hospTot = hospUn + hospVa

    # ax1.plot(curWeekNum,hospUn)
    # ax1.plot(curWeekNum,hospTot,'k')
    ax1.fill_between(curWeekNum,100*hospTot/hospTot,color='b',label='Vaccinerede')
    ax1.fill_between(curWeekNum,100*hospUn/hospTot,color='xkcd:light red',label='Uvaccinerede')
    # ax1.plot(rnTime(curWeekNum,5),rnMean(100*hospUn/hospTot,5),color='r')
    # ax1.plot(curWeekNum,hospVa/hospTot,color='b')
    # ax1.fill_between(curWeekNum,hospVa/hospTot,hospUn/hospTot,color='b')
    # ax2.fill_between(curWeekNum,agedfUnvacc['Befolkningstørrelse'].values,color='xkcd:light red')
    # ax2.fill_between(curWeekNum,agedfVacc['Befolkningstørrelse'].values,agedfUnvacc['Befolkningstørrelse'].values,color='b')

    popUn = agedfUnvacc['Befolkningstørrelse'].values
    popVa = agedfVacc['Befolkningstørrelse'].values
    popTot = popUn + popVa
    ax2.fill_between(curWeekNum,100*popTot/popTot,color='b')
    ax2.fill_between(curWeekNum,100*popUn/popTot,color='xkcd:light red')
    # ax1.plot(curWeekNum,agedfUnvacc['Antal nyindlagte'].values,color='xkcd:light red')
    # ax1.plot(curWeekNum,agedfVacc['Antal nyindlagte'].values,color='b')
    # ax1.stackplot(curWeekNum,
    #                 agedfUnvacc['Antal nyindlagte'],
    #                 agedfVacc['Antal nyindlagte'])

    # plt.plot(curWeekNum,curdf['Nyindlagte pr. 100.000'])
    # # plt.plot(curWeekNum,curdf['Befolkningstørrelse'])

    ax1.grid(axis='y')
    ax2.grid(axis='y')
    ax1.set_ylabel('Andel af alle indlæggelser [%]')
    ax2.set_ylabel('Andel af befolkningsstørrelse [%]')

    ax1.set_xlim([curWeekNum.values[0],curWeekNum.values[-1]])
    ax1.set_title(curAgeLabel)

    curMax = 100
    ax1.set_ylim(bottom=0,top=curMax)
    ax2.set_ylim(bottom=0,top=curMax)
    fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-205-c3c13671c328>:33: RuntimeWarning: invalid value encountered in true_divide
  ax1.fill_between(curWeekNum,100*hospTot/hospTot,color='b',label='Vaccinerede')
<ipython-input-205-c3c13671c328>:34: RuntimeWarning: invalid value encountered in true_divide
  ax1.fill_between(curWeekNum,100*hospUn/hospTot,color='xkcd:light red',label='Uvaccinerede')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [206]:

# curdf = df 
curdf = shortdf[(shortdf.Aldersgruppe == '80+') & (shortdf.Vaccinationsstatus == 'Ikke vaccineret')]
curdf

plt.figure()

# plt.plot(curdf.)

NameError: name 'shortdf' is not defined